## Problem Statement:
Use Large Language Models or a new programming approach to identify a Person, Place or Thing from a list of keywords while playing a 20 Questions game using a turn by turn agent approach.  

Possible Solutions:

* Large Languate Models can be fine tuned with landmark and city GIS coordinates centerpoints from Photography to better align questioning to Country, City, State and Landmark approximation.

* A phone Texts conversation datset with GIS coordinates could also potentially be used with converted locations to identify localization of languages, specific speach patterns or colloquialisms.

* For Thigs, product descriptions or product reviews can be used possibly.

* For Persons, Wikipedia can be used and any social networking like LinkedIn or Dating Sites.

* Other approaches without fine tunning is to identify tokens that will yield great search signals to segmentations for quickly arriving at a keyword within a 20 question range that can be reverse lookup unsepervised model based on current use model weights or embeddings.

Examples:
* https://www.cia.gov/the-world-factbook/
* https://public-nps.opendata.arcgis.com/datasets/nps::national-register-of-historic-places-points/explore


## Keywords List


In [ ]:
import pandas as pd
import json

from importlib.machinery import SourceFileLoader
keywords = SourceFileLoader("keywords",'/kaggle/input/llm-20-questions/llm_20_questions/keywords.py').load_module()
df = json.loads(keywords.KEYWORDS_JSON)
words = []
for c in df:
    print(c['category'], len(c['words']))
    for w in c['words']:
        words.append([c['category'], w["keyword"], w["alts"], "", "", 0.0, 0.0])
df = pd.DataFrame(words, columns=['Category','Word','Alternatives', "Cat1", "Cat2", "Lat", "Lon"])
df.tail()

## Template Formatting
* https://www.promptingguide.ai/models/gemma

In [ ]:
template_special_tokens = ['<bos>','<start_of_turn>user','<start_of_turn>model','<end_of_turn>','<eos>']
#Agent 1 - Guesser (Model?)
#Agent 2 - Answerer (User?)
#Agent 3 - Guesser (Model?)
#Agent 4 - Answerer (User?)

def agentx(obs, cfg):
    if obs.turnType == "ask": response = ""
    elif obs.turnType == "guess": response = ""
    elif obs.turnType == "answer": response = ""
    else: response = "Am I Halucinating?  **YES**"
    return response

## 20 Questions Submission Model

In [ ]:
%%bash
cd /kaggle/working
pip install -q -U -t /kaggle/working/submission/lib immutabledict sentencepiece
git clone https://github.com/google/gemma_pytorch.git > /dev/null
mkdir /kaggle/working/submission/lib/gemma/
mv /kaggle/working/gemma_pytorch/gemma/* /kaggle/working/submission/lib/gemma/

In [ ]:
%%writefile submission/main.py

import torch, itertools, contextlib
import os, sys, re, gc
from typing import Iterable
from pathlib import Path
gc.enable()

KAGGLE_AGENT_PATH = "/kaggle_simulations/agent/"
if os.path.exists(KAGGLE_AGENT_PATH):
    sys.path.insert(0, os.path.join(KAGGLE_AGENT_PATH, 'lib'))
    WEIGHTS_PATH = os.path.join(KAGGLE_AGENT_PATH, "gemma/pytorch/2b-it/2")
else:
    sys.path.insert(0, "/kaggle/working/submission/lib")
    WEIGHTS_PATH = "/kaggle/input/gemma/pytorch/2b-it/2"

from gemma.config import get_config_for_2b
from gemma.model import GemmaForCausalLM

#This can be condensed and complimented with game memory storage open('gamemaster.dat'), open('player.dat')
class AgentFormatter:
    def __init__(self, sp: str = None, fse: Iterable = None):
        self._system_prompt = sp
        self._few_shot_examples = fse
        self._turn_user = f"<start_of_turn>user\n{{}}<end_of_turn>\n"
        self._turn_model = f"<start_of_turn>model\n{{}}<end_of_turn>\n"
        self.reset()
    def __repr__(self):
        return self._state
    def user(self, prompt):
        self._state += self._turn_user.format(prompt)
        return self
    def model(self, prompt):
        self._state += self._turn_model.format(prompt)
        return self
    def reset(self):
        self._state = ""
        if self._system_prompt is not None:
            self.user(self._system_prompt)
        if self._few_shot_examples is not None:
            self.apply_turns(self._few_shot_examples, start_agent='user')
        return self
    def apply_turns(self, turns: Iterable, start_agent: str):
        formatters = [self.model, self.user] if start_agent == 'model' else [self.user, self.model]
        formatters = itertools.cycle(formatters)
        for fmt, turn in zip(formatters, turns):
            fmt(turn)
        return self

@contextlib.contextmanager
def _set_default_tensor_type(dtype: torch.dtype):
    torch.set_default_dtype(dtype)
    yield
    torch.set_default_dtype(torch.float)

class Agent:
    def __init__(self, sp=None, fse=None):
        self._device = torch.device('cuda:0')
        self.formatter = AgentFormatter(sp=sp, fse=fse)
        print("Initializing model")
        model_config = get_config_for_2b()
        model_config.tokenizer = WEIGHTS_PATH + '/tokenizer.model'
        with _set_default_tensor_type(model_config.get_dtype()):
            model = GemmaForCausalLM(model_config)
            model.load_weights(WEIGHTS_PATH + '/gemma-2b-it.ckpt')
            self.model = model.to(self._device).eval()
    def __call__(self, obs, *args):
        self._start_session(obs)
        prompt = str(self.formatter)
        response = self._call_llm(prompt)
        response = self._parse_response(response, obs)
        print(obs.turnType, response)
        return response
    def _call_llm(self, prompt, max_nt=40, **sampler_kwargs):
        if sampler_kwargs is None:
            sampler_kwargs = {'temperature': 0.9, 'top_p': 0.9, 'top_k': 50,}
        response = self.model.generate(
            prompt, device=self._device, output_len=max_nt, **sampler_kwargs,)
        return response
    def _parse_keyword(self, response: str):
        match = re.search(r"(?<=\*\*)([^*]+)(?=\*\*)", response) #Check keyword listing for closest match, update regex
        if match is None: keyword = ''
        else: keyword = match.group().lower()
        return keyword
    def _parse_response(self, response: str, obs: dict):
        if obs.turnType == 'ask':
            match = re.search(".+?\?", response.replace('*', ''))
            if match is None: question = "Is it a place?" #make random choice for person, place, thing
            else: question = match.group()
            return question
        elif obs.turnType == 'guess':
            guess = self._parse_keyword(response)
            if guess is None or len(guess) <= 1: 
                return "no guess" #changed to random keyword?
            else: 
                return guess
        elif obs.turnType == 'answer':
            answer = self._parse_keyword(response)
            return 'yes' if 'yes' in answer else 'no'
        else: raise ValueError("Unknown turn type:", obs.turnType)

def interleave_unequal(x, y):
    return [item for pair in itertools.zip_longest(x, y) for item in pair if item is not None]

class QuestionerAgent(Agent):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
    def _start_session(self, obs):
        self.formatter.reset()
        self.formatter.user("You are playing the 20 Questions game in the role of the Questioner.")
        turns = interleave_unequal(obs.questions, obs.answers)
        self.formatter.apply_turns(turns, start_agent='model')
        if obs.turnType == 'ask':
            self.formatter.user("It is your turn to ask a yes or no question to find the person, place or thing.")
        elif obs.turnType == 'guess':
            self.formatter.user("It is your turn to guess the person, place or thing. Surround your guess with double asterisks, like **yes** or **no**.")
    def _parse_response(self, response: str, obs: dict):
        if obs.turnType == 'ask':
            match = re.search(".+?\?", response.replace('*', ''))
            if match is None: question = "Is it a place?" #make random choice for person, place, thing
            else: question = match.group()
            return question
        elif obs.turnType == 'guess':
            guess = self._parse_keyword(response)
            if guess is None or len(guess) <= 1: 
                return "no guess" 
            else: 
                return guess
        elif obs.turnType == 'answer':
            answer = self._parse_keyword(response)
            return 'yes' if 'yes' in answer else 'no'
        else: raise ValueError("Unknown turn type:", obs.turnType)

class AnswererAgent(Agent):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
    def _start_session(self, obs):
        self.formatter.reset()
        self.formatter.user(f"You are playing the 20 Questions game in the role of the Answerer. The keyword is {obs.keyword} in the category {obs.category}.")
        turns = interleave_unequal(obs.questions, obs.answers)
        self.formatter.apply_turns(turns, start_agent='user')
        self.formatter.user(f"The question is about the keyword {obs.keyword} in the category {obs.category}. Give yes or no answer and surround your answer with double asterisks, like **yes** or **no**.")
    def _parse_response(self, response: str, obs: dict):
        answer = self._parse_keyword(response)
        return 'yes' if 'yes' in answer else 'no'

sp = "You are playing the 20 Questions game. In this game, the Answerer thinks of a keyword and responds to yes or no questions by the Questioner. The keyword is a specific person, place, or thing."
fse = [
    "You are playing the 20 Questions game in the role of the Questioner. Please ask your first question to help guess a person, place or thing keyword.",
    "Is it a person?", "**no**",
    "Is is a place?", "**yes**",
    "Is it a country?", "**yes** Now guess the keyword.",
    "**France**", "Correct!",
]
agent = None

def get_agent(name: str):
    global agent
    if agent is None and name == 'questioner':
        agent = QuestionerAgent(sp=sp, fse=fse,)
    elif agent is None and name == 'answerer':
        agent = AnswererAgent(sp=sp, fse=fse,)
    assert agent is not None, "Agent not initialized."
    return agent

def agent_fn(obs, cfg):
    if obs.turnType == "ask": response = get_agent('questioner')(obs)
    elif obs.turnType == "guess": response = get_agent('questioner')(obs)
    elif obs.turnType == "answer": response = get_agent('answerer')(obs)
    
    if response is None or len(response) <= 1: return "no" #changed default
    else: return response

# Testing

In [ ]:
!pip install -q pygame
!pip install -q 'kaggle_environments>=1.14.8'

In [ ]:
#Run Code
%run submission/main.py

In [ ]:
from kaggle_environments import make
env = make("llm_20_questions", debug=True)
agent = "/kaggle/working/submission/main.py"
env.reset()
logs = env.run([agent, agent, agent, agent])
#while not env.done: #add steps here for testing
env.render(mode="ipython", width=800, height=800)

# Package

In [ ]:
!apt install pigz pv > /dev/null

In [ ]:
!tar --use-compress-program='pigz --fast --recursive | pv' -cf submission.tar.gz -C /kaggle/working/submission . -C /kaggle/input/ gemma/pytorch/2b-it/2